In [3]:
from NN import Model

import torch
import torch.nn as nn

import torchvision
import torchvision.transforms as transforms

from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

## Creating custom dataset

In [4]:
TEST_DIR   = 'inputs/data/test'
BATCH_SIZE = 1
IMG_SIZE   = 128
CLASSES    = ('non_smoking', 'smoking')

TRANSFORM = transforms.Compose(
                [transforms.ToTensor(), #0-255 > 0-1, numpy to tensor
                transforms.Resize((IMG_SIZE, IMG_SIZE))
                ,transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))] #
            )

testloader = DataLoader(
                  torchvision.datasets.ImageFolder(TEST_DIR, transform=TRANSFORM),
                  batch_size=BATCH_SIZE, shuffle=False
                  )

test_size  = len(testloader)
test_size

500

In [5]:
NUM_CLASSES = len(CLASSES)

model = Model(NUM_CLASSES).to(device)

SAVED_MODEL_PATH = 'saved_models/using_testAcc0.78.model'
model.load_state_dict(torch.load(SAVED_MODEL_PATH, map_location=torch.device(device)))

<All keys matched successfully>

In [6]:
tp, tn, fp, fn = 0, 0, 0, 0

for img, label in tqdm(iter(testloader)):
    output = model(img).data
    out = torch.max(output, 1)[1]
    pred = CLASSES[out]
    
    if   out == 1 and label == 1: tp += 1 # True +ve
    elif out == 1 and label == 0: fp += 1 # False +ve
    elif out == 0 and label == 1: fn += 1 # False -ve
    elif out == 0 and label == 0: tn += 1 # True -ve
    
    # print(tp, tn, fp, fn, end='/r')

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:14<00:00, 34.39it/s]


In [7]:
print(tp, tn, fp, fn)

228 103 147 22


In [8]:
t = len(testloader)/100

In [9]:
print(tp/t, tn/t, fp/t, fn/t)

45.6 20.6 29.4 4.4
